Jérémie SCHEMITH

Suite au devoir 1 , nous avons un dataset clean et merged avec CountryGDP et CounrtyPopulation

# 2.1 - CLASSEUR BINAIRE

In [1]:
#Chargement des modules commun
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import *
from sklearn.compose import *

data = pd.read_csv('devoir1_df.csv')

data.head()

,age,pages,first_item_prize,gender,ReBuy,News_click,country,revenue,population,GDP_inhab
0,41.0,6.000000,28.0,Fem,False,4.0,China,113.0,1360720000,9055
1,34.0,4.000000,15.5,Fem,True,2.0,China,36.0,1360720000,9055
2,38.0,5.000000,42.0,Fem,False,7.0,China,111.0,1360720000,9055
3,20.0,5.725134,44.0,Fem,False,2.0,China,71.0,1360720000,9055
4,39.0,10.000000,10.0,Fem,True,4.0,China,80.0,1360720000,9055


Nous allons séparé nos données car le dataset est entir ci dessus

In [2]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(data, test_size=0.2, 
                                       random_state=42)

2.1.1 - Création de la fonction qui va transformer la variable revenue en variable binaire

In [3]:
def transformLabelIntoBinary(df: pd.DataFrame , targetedColumn='revenue') -> pd.DataFrame:
    targetedMean = df[targetedColumn].mean()
    new_df = df
    new_df[targetedColumn] = np.where( df[targetedColumn] > targetedMean , 1 , 0)
    return new_df

train_set = transformLabelIntoBinary(train_set)
train_set.head()

,age,pages,first_item_prize,gender,ReBuy,News_click,country,revenue,population,GDP_inhab
9254,36.0,6.0,10.0,Masc,False,3.0,Ghana,0,24658823,3316
1561,36.0,6.0,22.0,Masc,True,4.0,Bangladesh,0,152518015,1963
1670,38.0,3.0,44.0,Fem,False,3.0,Russia,0,143700000,17518
6087,36.0,4.0,15.5,Masc,False,10.0,Argentina,1,41660096,17917
6669,40.0,4.0,15.5,Fem,False,7.0,Sudan,0,37964000,2549


2.1.2 - Echantillonage aléatoire startifié

In [4]:
from sklearn.utils import resample

train_set_without_GDP_POP = train_set.drop(['population','GDP_inhab'],axis=1)
train_set_without_GDP = train_set.drop(['GDP_inhab'],axis=1)

train_set.count()

age                 8000
pages               8000
first_item_prize    8000
gender              8000
ReBuy               8000
News_click          8000
country             8000
revenue             8000
population          8000
GDP_inhab           8000
dtype: int64

In [5]:
# Ici nous pouvons choisir le nombre d'échantillons que l'on souhaite 2000, 4000,8000 ...
# je l'ai fait que pour 2000 car cela était fastidieux de plus les résultats ne changent pas

def getStartifiedResampled(df: pd.DataFrame,number_samples=2000, targetedColumn='revenue') -> pd.DataFrame:
    resampled_df = resample(df, n_samples=number_samples, replace=False, stratify=df[targetedColumn], random_state=5)
    return resampled_df

train_set_without_GDP_POP_resampled = getStartifiedResampled(train_set_without_GDP_POP)
train_set_without_GDP_resampled = getStartifiedResampled(train_set_without_GDP)
train_set_resampled = getStartifiedResampled(train_set)
train_set_resampled.count()

age                 2000
pages               2000
first_item_prize    2000
gender              2000
ReBuy               2000
News_click          2000
country             2000
revenue             2000
population          2000
GDP_inhab           2000
dtype: int64

2.1.3 - Training et test par Cross Validation

In [6]:
train_set_resampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 5965 to 6272
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   age               2000 non-null   float64
 1   pages             2000 non-null   float64
 2   first_item_prize  2000 non-null   float64
 3   gender            2000 non-null   object 
 4   ReBuy             2000 non-null   bool   
 5   News_click        2000 non-null   float64
 6   country           2000 non-null   object 
 7   revenue           2000 non-null   int32  
 8   population        2000 non-null   int64  
 9   GDP_inhab         2000 non-null   int64  
dtypes: bool(1), float64(4), int32(1), int64(2), object(2)
memory usage: 150.4+ KB


In [7]:
train_set_resampled_E = pd.get_dummies(train_set_resampled[train_set_resampled.columns[:-1]])


In [8]:
train_set_resampled_E.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 5965 to 6272
Data columns (total 59 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   age                     2000 non-null   float64
 1   pages                   2000 non-null   float64
 2   first_item_prize        2000 non-null   float64
 3   ReBuy                   2000 non-null   bool   
 4   News_click              2000 non-null   float64
 5   revenue                 2000 non-null   int32  
 6   population              2000 non-null   int64  
 7   gender_Fem              2000 non-null   bool   
 8   gender_Masc             2000 non-null   bool   
 9   country_Afghanistan     2000 non-null   bool   
 10  country_Algeria         2000 non-null   bool   
 11  country_Argentina       2000 non-null   bool   
 12  country_Australia       2000 non-null   bool   
 13  country_Bangladesh      2000 non-null   bool   
 14  country_Brazil          2000 non-null   bo

In [9]:
train_set_resampled_E['class'] = train_set_resampled_E['revenue']

In [10]:
X_train=train_set_resampled_E.iloc[:,:-1]
y_train=train_set_resampled_E['class']
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf= DecisionTreeClassifier(criterion="entropy")

cross_val_score(clf, X_train, y_train, cv=3, scoring="accuracy")


array([1., 1., 1.])

In [11]:
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, X_train, y_train, cv=3)

In [12]:
from sklearn import metrics
print(metrics.confusion_matrix(y_train,y_pred))

[[1195    0]
 [   0  805]]


In [13]:
# Automatisation de l'entrainement avec la validation croisée pour les autres échatillons
# R : Resampled / E: Categorical columns Encoded / K : variable petit k
def trainingAndCrossValidation(df: pd.DataFrame, k=3):
    df_E = pd.get_dummies(df[df.columns[:]])
    df_E['class'] = df_E['revenue']
    X_train=df_E.iloc[:,:-1]
    y_train=df_E['class']
    clf= DecisionTreeClassifier(criterion="entropy")
    cross_val_score(clf, X_train, y_train, cv=k, scoring="accuracy")
    y_pred = cross_val_predict(clf, X_train, y_train, cv=k)
    return X_train,y_train,y_pred

X_trainRE_K3,y_trainRE_K3,y_predRE_K3 = trainingAndCrossValidation(train_set_resampled)
X_trainRE_K10,y_trainRE_K10,y_predRE_K10 = trainingAndCrossValidation(train_set_resampled,k=10)

X_train_without_GDP_RE_K3,y_train_without_GDP_RE_K3,y_pred_without_GDP_RE_K3 = trainingAndCrossValidation(train_set_without_GDP_resampled)
X_train_without_GDP_RE_K10,y_train_without_GDP_RE_K10,y_pred_without_GDP_RE_K10 = trainingAndCrossValidation(train_set_without_GDP_resampled,k=10)

X_train_without_GDP_POP_RE_K3,y_train_without_GDP_POP_RE_K3,y_pred_without_GDP_POP_RE_K3 = trainingAndCrossValidation(train_set_without_GDP_POP_resampled)
X_train_without_GDP_POP_RE_K10,y_train_without_GDP_POP_RE_K10,y_pred_without_GDP_POP_RE_K10 = trainingAndCrossValidation(train_set_without_GDP_POP_resampled,k=10)



2.1.4 - Mesurer les métriques de précision, rappel et F1-Mesure

In [14]:

from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix

def print_scores(y_true, y_pred):
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    conf_matrix = confusion_matrix(y_true, y_pred)
    
    print(f'Precision: {precision}')
    print(f'Recall: {recall}')
    print(f'F1 Score: {f1}')
    print(f'Confusion Matrix:\n{conf_matrix}')

print_scores(y_trainRE_K3,y_predRE_K3)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1195    0]
 [   0  805]]


In [15]:
print_scores(y_trainRE_K10,y_predRE_K10)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1195    0]
 [   0  805]]


In [16]:
print_scores(y_train_without_GDP_RE_K10,y_pred_without_GDP_RE_K10)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1195    0]
 [   0  805]]


In [17]:
print_scores(y_train_without_GDP_RE_K3,y_pred_without_GDP_RE_K3)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1195    0]
 [   0  805]]


2.1.5 - Optimisation des hyper-paramètres 

In [18]:
from sklearn.model_selection import GridSearchCV

params = {'max_depth': list(range(2, 20)), 'min_samples_split': [5,10, 15, 20]}

def estimateBestParameters(params,X_train,y_train):
    grid_search_cv = GridSearchCV(DecisionTreeClassifier(criterion='entropy'), 
                                  params, n_jobs=-1, verbose=1, cv=10)
    grid_search_cv.fit(X_train, y_train)
    print(grid_search_cv.best_estimator_)
    return grid_search_cv

gridSearchtrainRE_K10 = estimateBestParameters(params, X_trainRE_K10, y_trainRE_K10)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_split=5)


In [19]:
gridSearchtrain_without_GDP_RE_K10 = estimateBestParameters(params, X_train_without_GDP_RE_K10, y_train_without_GDP_RE_K10)

Fitting 10 folds for each of 72 candidates, totalling 720 fits
DecisionTreeClassifier(criterion='entropy', max_depth=2, min_samples_split=5)


2.1.6 - Validation avec un Jeu de Test 

In [20]:
df_test = transformLabelIntoBinary(test_set)
df_test = pd.get_dummies(df_test[df_test.columns[:]])
df_test['class'] = df_test['revenue']
df_test['class'].value_counts(normalize=True)

class
0    0.576
1    0.424
Name: proportion, dtype: float64

In [21]:
X_test=df_test.iloc[:,:-1]
y_test=df_test['class']
y_pred = gridSearchtrainRE_K10.predict(X_test)
print_scores(y_pred,y_test)

Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1152    0]
 [   0  848]]


In [22]:
df_test = test_set.drop('GDP_inhab',axis=1)
df_test = transformLabelIntoBinary(df_test)
df_test = pd.get_dummies(df_test[df_test.columns[:]])
df_test['class'] = df_test['revenue']
X_test_without_GDP=df_test.iloc[:,:-1]
y_test_without_GDP= df_test['class']
y_pred_without_GDP = gridSearchtrain_without_GDP_RE_K10.predict(X_test_without_GDP)
print_scores(y_pred_without_GDP,y_test_without_GDP)


Precision: 1.0
Recall: 1.0
F1 Score: 1.0
Confusion Matrix:
[[1152    0]
 [   0  848]]


2.1.7 - Tableaux

In [23]:
print(metrics.classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1152
           1       1.00      1.00      1.00       848

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



In [24]:
print(metrics.classification_report(y_test_without_GDP,y_pred_without_GDP))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1152
           1       1.00      1.00      1.00       848

    accuracy                           1.00      2000
   macro avg       1.00      1.00      1.00      2000
weighted avg       1.00      1.00      1.00      2000



# 2.2 - CLASSEUR MULTI-CLASS

2.2.1 - Vous devez écrire une fonction pour remplacer cette variable « revenu » par une 
variable multi-classes selon les trois catégories de revenu

In [25]:
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder

train_set, test_set = train_test_split(data, test_size=0.2, 
                                       random_state=42)

def transformLabelIntoMultiClass(df: pd.DataFrame):
    X = df.drop('revenue', axis=1)
    y = df['revenue']
    
    # Apply the transformation to make the distribution more normal
    pt = PowerTransformer()
    y_transformed = pt.fit_transform(y.values.reshape(-1, 1))
    
    # Calculate the mean (M) and standard deviation (E) of the new distribution
    M = y_transformed.mean()
    E = y_transformed.std()
    
    min_trshld = M - 2 * E
    max_trshld = M + 2 * E
    print(f"Tresholds {min_trshld} | {max_trshld}")
    # Use np.where to create a new column 'revenue_category' based on conditions
    df['revenue'] = np.where(y_transformed < min_trshld, 0,
                                      np.where(y_transformed > max_trshld, 2, 1))
    
    return df

train_set = transformLabelIntoMultiClass(train_set)
print('0 for low revnue ::: 1 for mid_revenue ::: 2 for high_revenue')
train_set['revenue'].value_counts()


Tresholds -2.0 | 2.0
0 for low revnue ::: 1 for mid_revenue ::: 2 for high_revenue


revenue
1    7736
0     157
2     107
Name: count, dtype: int64

2.2.2 - Quel est le meilleur classeur pouvant discriminer les 3 revenus et justifiez votre 
réponse

Nous allons tester avec k=3 premièrement

In [ ]:
train_set = pd.get_dummies(train_set[train_set.columns[:-1]])
X_train=train_set.iloc[:,:-1]
train_set['class'] = train_set['revenue']
y_train=train_set['class']
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf= DecisionTreeClassifier(criterion="entropy")

cross_val_score(clf, X_train, y_train, cv=3, scoring="accuracy")
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, X_train, y_train, cv=3)
print(metrics.confusion_matrix(y_train,y_pred))

In [ ]:
test_set = transformLabelIntoMultiClass(test_set)
test_set['revenue'].value_counts()

In [ ]:
test_set = pd.get_dummies(test_set[test_set.columns[:-1]])
X_test=test_set.iloc[:,:-1]
test_set['class'] = test_set['revenue']
y_test=test_set['class']
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf= DecisionTreeClassifier(criterion="entropy")

cross_val_score(clf, X_test, y_test, cv=3, scoring="accuracy")
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, X_test, y_test, cv=3)
print(metrics.confusion_matrix(y_test,y_pred))

Faisons avec k=10 :

In [ ]:
train_set = pd.get_dummies(train_set[train_set.columns[:-1]])
X_train=train_set.iloc[:,:-1]
train_set['class'] = train_set['revenue']
y_train=train_set['class']
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf= DecisionTreeClassifier(criterion="entropy")

cross_val_score(clf, X_train, y_train, cv=3, scoring="accuracy")
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, X_train, y_train, cv=10)
print(metrics.confusion_matrix(y_train,y_pred))

In [ ]:
test_set = transformLabelIntoMultiClass(test_set)
test_set['revenue'].value_counts()

In [ ]:
test_set = pd.get_dummies(test_set[test_set.columns[:-1]])
X_test=test_set.iloc[:,:-1]
test_set['class'] = test_set['revenue']
y_test=test_set['class']
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
clf= DecisionTreeClassifier(criterion="entropy")

cross_val_score(clf, X_test, y_test, cv=10, scoring="accuracy")
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(clf, X_test, y_test, cv=10)
print(metrics.confusion_matrix(y_test,y_pred))

On choisit k=10 car nous n'avons pas de FP/FN quelques soit k

2.2.3 - Comparer les résultats obtenus avec ceux du classeur binaire 

Si la préocupation principale est de simplement distinguer les revenus bas des revenus élevés, 
un modèle binaire pourrait suffire, et il pourrait être plus simple et plus interprétable.

Si l on souhaite également identifier une catégorie de revenu intermédiaire (revenu-moyen), 
un modèle multi-classes est plus approprié, car il peut fournir une classification plus fine